In [15]:
import ijson
import json
from pathlib import Path
from decimal import Decimal

In [6]:
N_USERS=100
selected_users = []
selected_user_ids = set()

with open("//Users//yesh//Documents//BotShield//TwiBot-22//user_det.json", "rb") as f:
    for user in ijson.items(f, "item"):
        raw_id = user.get("id")
        if raw_id is None:
            continue

        # Normalize user ID: "u123..." → 123...
        user_id = int(raw_id.lstrip("u"))
        print(user_id)

        user["id"] = user_id  # overwrite with normalized ID

        selected_users.append(user)
        selected_user_ids.add(user_id)

        if len(selected_users) >= N_USERS:
            break


1217628182611927040
2664730894
1266703520205549568
1089159225148882949
36741729
1365527332627247104
1679822588
1519144464
15211869
1309034737756000256
1341789703633178624
848975226
1109809482656149504
138814032
457554412
17899123
2465283662
1467973039883182090
234059290
1142890104853106688
779266494270173187
1275028426672549888
284870222
1280925874267787270
2599121383
912974971370115072
1322906536029626368
83389771
129211757
765845
804075344
935031277
14788268
3212706501
2496941382
21920591
213809534
2196860159
1099603759259238400
1213172638790512642
177622514
10129132
1478784043105894403
864305332075364352
1293469199554285568
1141322718559666176
29458079
3304925983
229389391
1475101296520830982
1459787141928660998
871016931213619200
771758047412748288
1482043516620525579
933227158398386182
49009937
1360303708777635844
1433818532215730176
1484114013566943232
196278935
1457083371146915842
731478516181565440
383950602
13439042
1009495181882773506
73710441
542721191
820367095120547841
233

In [7]:
print(selected_user_ids)

{1217628182611927040, 1266703520205549568, 1365527332627247104, 1309034737756000256, 1341789703633178624, 1089159225148882949, 1109809482656149504, 1142890104853106688, 779266494270173187, 1275028426672549888, 1467973039883182090, 1280925874267787270, 1475101296520830982, 1482043516620525579, 1459787141928660998, 933227158398386182, 1519144464, 1478784043105894403, 1484088723730055170, 280747022, 1395509316753534987, 234059290, 98804784, 22396466, 13439042, 2465283662, 284870222, 138814032, 229389391, 36741729, 17899123, 912974971370115072, 1293469199554285568, 1009804608540200960, 820367095120547841, 1009495181882773506, 1192861100804575234, 196278935, 86020763, 29458079, 1353063402160099335, 542721191, 14788268, 15918780, 3212706501, 20813318, 10129132, 1679822588, 2196860159, 1162025098230095872, 1322906536029626368, 1290233331683008512, 1483049550764187648, 1438341642, 864305332075364352, 241877255, 383950602, 1031077915188846594, 2664730894, 35708687, 49009937, 1360303708777635844

In [8]:
with open("subset_users.json", "w") as f:
    json.dump(selected_users, f)

with open("selected_user_ids.json", "w") as f:
    json.dump(list(selected_user_ids), f)


In [16]:
def convert_decimals(obj):
    if isinstance(obj, Decimal):
        if obj % 1 == 0:
            return int(obj)
        else:
            return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_decimals(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_decimals(v) for v in obj]
    else:
        return obj

In [17]:
with open("selected_user_ids.json", "r") as f:
    selected_user_ids = set(json.load(f))

tweet_dir = Path("../TwiBot-22")
tweet_files = sorted(tweet_dir.glob("tweet_*.json"))
# print(len(tweet_files))
output = open("tweet_subset.json", "w")
output.write("[\n")

first = True
kept = 0

for tweet_file in tweet_files:
    print(f"Processing {tweet_file}")

    with open(tweet_file, "rb") as f:
        for tweet in ijson.items(f, "item"):
            author_id = tweet.get("author_id")
            # print(author_id)
            # exit
            if author_id is None:
                continue

            # author_id is already int → direct match
            if author_id in selected_user_ids:
                tweet = convert_decimals(tweet)
                if not first:
                    output.write(",\n")
                else:
                    first = False

                json.dump(tweet, output)
                kept += 1

output.write("\n]")
output.close()

print(f"Saved {kept} tweets to tweet_subset.json")

Processing ../TwiBot-22/tweet_0-010.json
Processing ../TwiBot-22/tweet_1-007.json
Processing ../TwiBot-22/tweet_2-003.json
Processing ../TwiBot-22/tweet_3-006.json
Processing ../TwiBot-22/tweet_4-004.json
Processing ../TwiBot-22/tweet_5-011.json
Processing ../TwiBot-22/tweet_6-008.json
Processing ../TwiBot-22/tweet_7-001.json
Processing ../TwiBot-22/tweet_8-005.json
Saved 62066 tweets to tweet_subset.json


In [18]:
tweets = json.load(open("tweet_subset.json"))

assert all(t["author_id"] in selected_user_ids for t in tweets)
print("All tweets belong to selected users")


All tweets belong to selected users
